In [1]:
import pandas as pd
import numpy as np

import numpy as np
import random
import random

random.seed(42)

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:
# import pandas as pd
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

start=2
end=3

In [4]:
#import lightgbm as lgb
from sklearn.linear_model import Lasso
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import joblib

# train, valid

## 1년후매출액증가율_20%

In [7]:
train = pd.read_csv('./data/class_0906_18학습_18추론.csv').drop(columns=['br_num_year', 'logit'])
test = pd.read_csv('./data/class_0906_18학습_19추론.csv').drop(columns=['br_num_year', 'logit'])
# 년도별 분석은 데이터셋만 바꿔서 동일하게 진행

X_train = train.drop(columns=['label','label_1','label_2'])
y_train = train['label_2']

X_test = test.drop(columns=['label','label_1','label_2'])
y_test = test['label_2']

## 1년후매출액증가율_상위1%

In [ ]:
train = pd.read_csv('./data/class_0906_18학습_18추론.csv').drop(columns=['logit','br_num_year'])
train.sort_values(by='label', ascending=False,inplace=True)
train.iloc[:int(len(train)*0.01),-5] = 1
train.iloc[int(len(train)*0.01):,-5] = 0

test = pd.read_csv('./data/class_0906_18학습_19추론.csv').drop(columns=['logit','br_num_year'])
test.sort_values(by='label', ascending=False,inplace=True)
test.iloc[:int(len(train)*0.01),-5] = 1
test.iloc[int(len(train)*0.01):,-5] = 0

X_train = train.drop(columns=['label','label_1','label_2'])
y_train = train['label']

X_test = test.drop(columns=['label','label_1','label_2'])
y_test = test['label']

## 3년후매출액증가율_72.8%

In [ ]:
train = pd.read_csv('./data/0910_18학습_18추론.csv').drop(columns=['br_num_year','logit'])
train = train[(train['3년매출액증가율'] < 100000) & (train['3년매출액증가율'] > -1000)]
train['label_3'] = train['3년매출액증가율'].apply(lambda x : 1 if x>=72.8 else 0)
test = pd.read_csv('./data/0910_18학습_19추론.csv').drop(columns=['br_num_year','logit'])
test = test[(test['3년매출액증가율'] < 100000) & (test['3년매출액증가율'] > -1000)]
test['label_3'] = test['3년매출액증가율'].apply(lambda x : 1 if x>=72.8 else 0)

X_train = train.drop(columns=['label_3','3년매출액증가율'])
y_train = train['label_3']

X_test = test.drop(columns=['label_3','3년매출액증가율'])
y_test = test['label_3']

## 3년후매출액증가율_상위1%

In [ ]:
train = pd.read_csv('./data/0910_18학습_18추론.csv').drop(columns=['br_num_year', 'logit'])
train = train[(train['3년매출액증가율'] < 100000) & (train['3년매출액증가율'] > -1000)]
train.sort_values(by='3년매출액증가율', ascending=False,inplace=True)
train.iloc[:int(len(train)*0.01),-1] = 1
train.iloc[int(len(train)*0.01):,-1] = 0
test = pd.read_csv('./data/0910_18학습_19추론.csv').drop(columns=['br_num_year', 'logit'])
test = test[(test['3년매출액증가율'] < 100000) & (test['3년매출액증가율'] > -1000)]
test.sort_values(by='3년매출액증가율', ascending=False,inplace=True)
test.iloc[:int(len(test)*0.01),-1] = 1
test.iloc[int(len(test)*0.01):,-1] = 0

X_train = train.drop(columns='3년매출액증가율')
y_train = train['3년매출액증가율']

X_test = test.drop(columns='3년매출액증가율')
y_test = test['3년매출액증가율']

In [8]:
test

,매출액,인건비,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,배당성향,...,자기자본증가율,자본금회전율,재고자산회전율1,총자본회전율,총자산증가율,label,매출액증가율_lag1,매출액증가율_lag2,label_1,label_2
0,17.134083,17.883560,23.90,0.00,0.00,20.55,-1.33,0.00,59.09,105.520000,...,11.46,9.07,136.658409,0.18,3.73,-9.30,-2.68,-3.15,0,0
1,16.999094,17.857230,3.07,1.21,8.95,86.04,-96.49,40.76,0.04,5.157305,...,505.16,0.14,0.570000,0.02,262.64,9912.11,-65.98,-30.98,1,1
2,18.811483,18.070176,4.76,1.69,1.46,166.83,0.78,15.35,4.42,0.000000,...,1.00,46.26,11.070000,0.25,3.00,-16.04,-5.38,-8.81,0,0
3,18.099159,17.889206,10.95,1.71,0.31,146.35,0.30,94.65,5.26,5.157305,...,-9.45,257.13,5.690000,1.55,38.27,-8.47,-11.09,6.86,0,0
4,17.100076,17.844010,36.00,1.13,0.27,193.05,-6.95,76.07,7.16,0.000000,...,0.28,11.24,13.210000,1.67,5.29,-44.23,62.20,25.90,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37571,16.973236,17.828264,4.86,1.79,0.44,352.18,-8.76,69.89,2.96,0.000000,...,-9.05,1.64,136.658409,1.19,-11.09,-9.35,0.44,-5.80,0,0
37572,17.095360,17.831986,10.11,2.24,1.27,152.14,638.57,85.75,4.00,0.000000,...,55.11,21.97,47.380000,1.54,20.69,15.38,-80.93,21.63,1,0
37573,16.941687,17.828971,100.00,2.47,0.27,561.93,320.80,80.65,11.45,0.000000,...,78.92,17.07,10.150000,4.89,19.53,-47.08,-73.39,38.40,0,0
37574,16.953155,17.841216,17.42,4.14,2.93,581.65,-8.83,20.90,4.43,0.000000,...,10.02,3.58,136.658409,0.93,-9.46,-9.30,47.39,32.24,0,0


In [9]:
test['label_2'].value_counts()/len(test)

0    0.764424
1    0.235576
Name: label_2, dtype: float64

In [10]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 학습

In [11]:
# from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

In [12]:
# NaN 값이 있는지 확인
print(train.isnull().sum())  # NaN 값의 개수를 확인

# NaN 값이 있으면 해당 행을 삭제
X_train = X_train[~y_train.isna()]
y_train = y_train[~y_train.isna()]

매출액            0
인건비            0
감가상각율          0
금융비용총부채        0
금융비용총비용비율      0
당좌비율           0
매출액증가율         0
매출원가율          0
매출채권회전율        0
배당성향           0
부채비율           0
비유동자산비율        0
비유동자산장기적합율     0
비유동자산회전율       0
수지비율           0
영업이익증가율        0
유형자산증가율        0
유형자산회전율        0
자기자본증가율        0
자본금회전율         0
재고자산회전율1       0
총자본회전율         0
총자산증가율         0
label          0
매출액증가율_lag1    0
매출액증가율_lag2    0
label_1        0
label_2        0
dtype: int64


# 하이퍼파라미터 튜닝

In [13]:
params = {
    'n_estimators':(100, 1000),
    'max_depth' : (3, 30),
    'min_samples_leaf' : (4, 30),
    'min_samples_split' : (4, 30),
    'max_features' : ['sqrt'],
    
}
rf_run = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_run, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 1000}
최적 예측 정확도: 0.7578


In [14]:
rf_run = RandomForestClassifier(random_state=42, max_depth=30, max_features='sqrt', min_samples_leaf=4, min_samples_split=4,n_estimators=1000)
rf_run.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, min_samples_leaf=4, min_samples_split=4,
                       n_estimators=1000, random_state=42)

In [15]:
y_test_pred = rf_run.predict(X_test) # X_test: 실제값 y_test_pred_prob: 예측 확률값 

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print(f'ACC: {accuracy_score(y_test,y_test_pred)}')
print(f'F1-score: {f1_score(y_test,y_test_pred)}')

ACC: 0.7748030657866724
F1-score: 0.21254420249395126


In [17]:
result= pd.DataFrame({'label':y_test, 'prediction_label':y_test_pred})
result

,label,prediction_label
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
37571,0,0
37572,0,0
37573,0,0
37574,0,0


# Response, Lift 도출

In [18]:
# 240724 - 1로 예측한 확률로 나래비 세우고 Response, Lift 도출

import pandas as pd
import numpy as np

# 예측 확률 뽑기
# rf_run : 랜덤포레스트 모델 이름
# X_test : 추론 입력 데이터
y_test_pred_prob = rf_run.predict_proba(X_test)

# y_test : 추론 데이터 label
# y_test_pred = 추론 데이터 prediction label
# predict_proba_0 : 0으로 예측한 확률
# predict_proba_1 : 1로 예측한 확률
result = pd.DataFrame({'label':list(y_test),'predict':list(y_test_pred),'predict_proba_0':list(y_test_pred_prob[:,0]),'predict_proba_1':list(y_test_pred_prob[:,1])})
a = result['label'].value_counts()/len(result)
base = a[1]
df = result

def calculate_hit(original_list, predict_list):
    original_set = set(original_list)
    predict_set = set(predict_list)
    
    hits = len(original_set & predict_set)
    
    return hits


label_ratio = 0.01
stupid = np.arange(0.01, 1.01, 0.01)
predict_ratio_list = [round(x,2) for x in stupid]

length_100 = int(round(len(df)*0.01,0))

start = 0
recall_li = []
precision_li = []
lift_li = []

for pre_ratio in predict_ratio_list:
    
    ori_length = int(round(len(df) * label_ratio,0))        
    pre_len = int(round(len(df) * pre_ratio,0))

    predict_list = df.sort_values(by=['predict_proba_1'], axis=0, ascending=False).index

    hits_pre = calculate_hit(df[df['label']==1].index, predict_list[start:pre_len])
    sectionlen = len(predict_list[start:pre_len])
    precision = round(hits_pre/sectionlen,2)
    # Precision 저장
    precision_li.append(precision)

    # Lift 저장
    lift = round(precision/base,2)
    lift_li.append(lift)

    start = start + length_100

df = pd.DataFrame({'Response': precision_li, 'Lift': lift_li})
df.to_csv(f'./결과1/240930_1년매출액증가율_20%_1819_RF.csv', index=False)
df

,Response,Lift
0,0.77,3.27
1,0.63,2.67
2,0.55,2.33
3,0.54,2.29
4,0.52,2.21
...,...,...
95,0.09,0.38
96,0.09,0.38
97,0.06,0.25
98,0.06,0.25


In [19]:
# 예측 및 확률값 계산
y_test_pred = rf_run.predict(X_test)
y_test_pred_prob = rf_run.predict_proba(X_test)  # 각 클래스에 대한 예측 확률

# 결과 데이터프레임 생성
result = pd.DataFrame({
    'label': y_test,
    'prediction_label': y_test_pred,
    'predict_proba1': y_test_pred_prob[:, 1],  # 클래스 1에 대한 예측 확률
})

result.to_csv(f'./결과2/240930_1년매출액증가율_20%_1819_RF.csv', index=False)
result

,label,prediction_label,predict_proba1
0,0,0,0.123545
1,1,1,0.620860
2,0,0,0.084485
3,0,0,0.210395
4,0,0,0.242756
...,...,...,...
37571,0,0,0.210226
37572,0,0,0.292604
37573,0,0,0.323625
37574,0,0,0.245405
